In [2]:
import ee
import geemap
import os

In [206]:
#Only have to do this once
ee.Authenticate()

#Not sure this is necessary every time
ee.Initialize()

Enter verification code: 4/0QFRh7h8C3a1AnL1RCbV8raixULcIeNCpSRSdU7zjyH-vpQUmFo-Pjk

Successfully saved authorization token.


In [3]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [4]:
#Create a point for EC Flux Tower
brma2 = ee.Geometry.Point(-60.20929722, -2.609097222)

#Determine the area I want my square buffer to be; can use ee.Geometry.Rectangle
areaM2 = 3.087969e9
square = brma2.buffer(ee.Number(areaM2).sqrt().divide(2), 1).bounds()

#Add the polygon to the map
Map.addLayer(square, {}, 'Manauas Buffer')

In [7]:
#Create an NDRE function
#There are four red edge bands (20 m) to choose from, I just picked the first
def my_fun(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

#Pull the Sentinel data, clip images to the square, and run NDRE function on all images
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2015-01-01', '2015-01-31') 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(square))
    .map(my_fun).select('NDRE'))

#Set visual paramters for NDRE
ndreParams={'min': 0.0,
            'max': 1.0,
            'palette' : ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
           }

#Calculate the median NDRE and add to the map
median = sentinel2.median()
Map.addLayer(median, ndreParams, 'Sentinel Median NDRE')

In [5]:
#Put the EC Tower on top of previous layer
Map.addLayer(brma2, {'color': 'FF0000', 'size': 50}, 'Manauas EC Flux Location')

In [69]:
#print(sentinel2.aggregate_array("system:time_start").getInfo())
#Above code times out. Too many images I guess.

In [64]:
#Exported for good fun; way to set palette for export? Directory is hard coded.
out_dir = '/Users/morgansteckler/Desktop/'
filename = os.path.join(out_dir, 'sentinel_ndre.tif')
geemap.ee_export_image(median, filename = filename, scale = 90, region = square, file_per_band=False)

Generating URL ...
An error occurred while downloading.
'<' not supported between instances of 'builtin_function_or_method' and 'builtin_function_or_method'
